In [ ]:
import numpy as np
from PIL import Image

def encrypt_block_cbc(block, hybrid_keys, sbox, previous_block):
    flat_block = block.flatten()

    xor_block = flat_block ^ previous_block.flatten()

    permutation_indices = np.argsort(hybrid_keys[:len(flat_block)])
    permuted_block = xor_block[permutation_indices]
    substituted_block = sbox[permuted_block % len(sbox)]

    return substituted_block.reshape(block.shape), substituted_block.flatten(), permutation_indices

def decrypt_block_cbc(block, previous_block, permutation_indices, sbox_inv):
    flat_block = block.flatten()

    # Reverse substitution
    substituted_block = np.array([sbox_inv[pixel] for pixel in flat_block])

    # Reverse permutation
    reverse_indices = np.argsort(permutation_indices)
    permuted_block = substituted_block[reverse_indices]

    # XOR with previous block
    decrypted_block = permuted_block ^ previous_block.flatten()

    return decrypted_block.reshape(block.shape)

def main(image_array):
    # Load image

    # Determine block size dynamically
    block_size = determine_block_size(image_array.shape)  # Assume this function is defined
    
    print(f"Using block size: {block_size}x{block_size}")

    # Divide image into blocks
    blocks, original_shape, padded_shape = divide_into_blocks(image_array, block_size)  # Assume this function is defined

    iv = np.random.randint(0, 256, (block_size, block_size, image_array.shape[2]), dtype=np.uint8)

    encrypted_blocks = []
    permutation_indices_list = []
    characteristics_list = []
    
    previous_block = iv  # Initialization Vector for CBC
    
    for block in blocks:
        # Compute block characteristics
        avg_intensity, variance, entropy = compute_image_characteristics(block)  # Assume this function is defined
        characteristics_list.append((avg_intensity, variance, entropy))

        # Generate keys and S-Box
        chen_keys = generate_chen_keys(block.size, avg_intensity, variance)  # Assume this function is defined
        lorenz_keys = generate_lorenz_keys(block.size, avg_intensity, entropy)  # Assume this function is defined
        logistic_keys = generate_logistic_keys(block.size, variance, entropy)  # Assume this function is defined
        hybrid_keys = (lorenz_keys + logistic_keys + chen_keys) % 256

        sbox = generate_dynamic_sbox(256, hybrid_keys)  # Assume this function is defined
        sbox_inv = np.argsort(sbox)

        encrypted_block, previous_block_flattened, permutation_indices = encrypt_block_cbc(
            block, hybrid_keys, sbox, previous_block
        )
        encrypted_blocks.append(encrypted_block)
        permutation_indices_list.append(permutation_indices)
        previous_block = previous_block_flattened.reshape(block.shape)  # Update to the current encrypted block

    encrypted_image = merge_blocks(encrypted_blocks, original_shape, padded_shape, block_size)  # Assume this function is defined
    # image_to_audio(encrypted_image, "output_audio1.wav")  # Assume this function is defined

    # extracted_image_array = audio_to_image("output_audio1.wav", encrypted_image.shape)  # Assume this function is defined

    # Divide the extracted image into blocks
    extracted_blocks, _, _ = divide_into_blocks(encrypted_image, block_size)

    decrypted_blocks = []
    previous_block = iv  # Start with IV again
    for i, block in enumerate(extracted_blocks):
        # Use stored characteristics for decryption
        avg_intensity, variance, entropy = characteristics_list[i]

        chen_keys = generate_chen_keys(block.size, avg_intensity, variance)
        lorenz_keys = generate_lorenz_keys(block.size, avg_intensity, entropy)
        logistic_keys = generate_logistic_keys(block.size, variance, entropy)
        hybrid_keys = (lorenz_keys + logistic_keys + chen_keys) % 256

        sbox = generate_dynamic_sbox(256, hybrid_keys)
        sbox_inv = np.argsort(sbox)

        decrypted_block = decrypt_block_cbc(block, previous_block, permutation_indices_list[i], sbox_inv)
        decrypted_blocks.append(decrypted_block)
        previous_block = block  # Update to the current ciphertext block

    decrypted_image = merge_blocks(decrypted_blocks, original_shape, padded_shape, block_size)  # Assume this function is defined

    # plot_images(image_array, encrypted_image, decrypted_image,image_array.shape)  # Assume this function is defined
    return encrypted_image,decrypted_image